In [1]:
from Cuttings import *

import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time
import copy
import PIL.Image as Image
import random

import torch.optim as optim
import torchvision.transforms as tf
from torch.utils import data 
import torch.nn as nn
import torch.nn.functional as F

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

import pickle 

In [2]:
# torchvision for pre-trained models
from torchvision import models

#### Classes

In [3]:
class MinMaxNormalization(object):
    """
    Normalized (Min-Max) the image.
    """
    def __init__(self, vmin=0, vmax=1):
        """
        Constructor of the grayscale transform.
        ----------
        INPUT
            |---- vmin (float / int) the desired minimum value.
            |---- vmax (float / int) the desired maximum value.
        OUTPUT
            |---- None
        """
        self.vmin = vmin
        self.vmax = vmax

    def __call__(self, image, mask=None):
        """
        Apply a Min-Max Normalization to the image.
        ----------
        INPUT
            |---- image (PIL.Image) the image to normalize.
        OUTPUT
            |---- image (np.array) the normalized image.
        """
        arr = np.array(image).astype('float32')
        arr = (arr - arr.min()) / (arr.max() - arr.min())
        arr = (self.vmax - self.vmin) * arr + self.vmin
        return arr

In [4]:
class Cuttings_Dataset(data.Dataset):
    def __init__(self, sample_df, data_path, data_augmentation=True):
        """
        Constructor of the dataset.
        """
        data.Dataset.__init__(self)
        self.sample_df = sample_df
        self.data_path = data_path
        self.data_augmentation = data_augmentation
        
        self.transform =  tf.Compose([tf.Grayscale(num_output_channels=3),
                                        tf.Resize(224),
                                        MinMaxNormalization(),
                                        tf.ToTensor(),
                                        tf.Normalize((0.5205568,0.5205568,0.5205568),(0.33119723,0.33119723,0.33119723))])
        
        if data_augmentation:
            self.transform = tf.Compose([tf.Grayscale(num_output_channels=3),
                                            tf.Resize(224),
                                            tf.RandomVerticalFlip(p=0.5),
                                            tf.RandomHorizontalFlip(p=0.5),
                                            tf.RandomRotation([-90,90],resample=False, expand=False, center=None, fill=None),
                                            MinMaxNormalization(),
                                            tf.ToTensor(),
                                            tf.Normalize((0.5205568,0.5205568,0.5205568),(0.33119723,0.33119723,0.33119723))])
        
    def __len__(self):
        """
        Get the number of samples in the dataset.
        """
        return self.sample_df.shape[0]


    def __getitem__(self, idx):
        """
        Get an item from the dataset.
        """
        # load image
        im = Image.open(self.data_path + self.sample_df.loc[idx,'path'])
        # load label
        label = torch.tensor(self.sample_df.loc[idx,'rock_type'])
        
        im = self.transform(im)
        return im,label

#### Loading data

In [5]:
classes = np.array(['ML', 'MS','BL','GN','OL'])

date = '05_05_20'

df = pd.read_csv('train/csv_'+date+'/train'+date+'_final.csv',index_col=0)

train,val = train_test_split(df,test_size=0.2,random_state=0,stratify=df['rock_type'])
cuttings_datasets = {}

cuttings_datasets['train'] = Cuttings_Dataset(train.reset_index(drop=True),'',data_augmentation=True)
cuttings_datasets['val'] = Cuttings_Dataset(val.reset_index(drop=True),'',data_augmentation=False)
cuttings_datasets['test'] = Cuttings_Dataset(pd.read_csv('test'+'/csv_'+date+'/'+'test'+date+'_final.csv',index_col=0),'',data_augmentation=False)

dataloaders = {x: torch.utils.data.DataLoader(cuttings_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=0)
              for x in ['train', 'val', 'test']}

dataset_sizes = {x: len(cuttings_datasets[x]) for x in ['train','val','test']}

In [6]:
# Flag for feature extracting. When False, we finetune the whole model,
# when True we only update the reshaped layer params

In [7]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [8]:
def initialize_model(num_classes, feature_extract=True, use_pretrained=True):
    model_ft = models.resnet18(pretrained=True)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Linear(num_ftrs, num_classes)
    return model_ft

In [9]:
model_ft = initialize_model(len(classes))

In [10]:
for name,param in model_ft.named_parameters():
    if param.requires_grad == True:
        print("\t",name)

	 fc.weight
	 fc.bias


In [11]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    loss_train = []
    acc_train = []
    loss_val = []
    acc_val = []
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        running_loss_train = 0.0
        running_corrects_train = 0
            
        model.train()# Set model to training mode
        
        # Iterate over data Training
        for inputs, labels in dataloaders['train']:
            inputs = inputs.to(device)
            inputs.require_grad = True
            labels = labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            outputs = model(inputs)
            preds = torch.argmax(outputs, 1)
            loss = criterion(outputs, labels)

            # backward + optimize
            loss.backward()
            optimizer.step()

            # statistics
            running_loss_train += loss.item() * inputs.size(0)
            running_corrects_train += torch.sum(preds == labels.data)
            
        epoch_loss_train = running_loss_train / dataset_sizes["train"]
        epoch_acc_train = running_corrects_train.double() / dataset_sizes["train"]
        loss_train.append(epoch_loss_train)
        acc_train.append(epoch_acc_train)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Train", epoch_loss_train, epoch_acc_train))
        
        running_loss_val = 0.0
        running_corrects_val = 0
            
        model.eval() # Set model to evaluate mode
        
        # Iterate over data Evaluation
        for inputs, labels in dataloaders["val"]:
            inputs = inputs.to(device)
            labels = labels.to(device)
                
            with torch.no_grad():
                # forward
                outputs = model(inputs)
                preds = torch.argmax(outputs, 1)
                loss = criterion(outputs, labels)   

            # statistics
            running_loss_val += loss.item() * inputs.size(0)
            running_corrects_val += torch.sum(preds == labels.data)

        epoch_loss_val = running_loss_val / dataset_sizes["val"]
        epoch_acc_val = running_corrects_val.double() / dataset_sizes["val"]
        loss_val.append(epoch_loss_val)
        acc_val.append(epoch_acc_val)
        
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                "Val", epoch_loss_val, epoch_acc_val))
            
        # deep copy the model
        if epoch_acc_val > best_acc:
            best_acc = epoch_acc_val
            best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, loss_train, acc_train, loss_val, acc_val

In [12]:
# prediciton
def prediciton(model):
    preds_vec = []
    true_vec = []
    
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloaders['test']:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            
            preds_vec+=preds.tolist()
            true_vec+=labels.tolist()
    return preds_vec, true_vec

In [13]:
date = '30_06_20'

In [14]:
# Get Cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Train Model
list_loss_train = []
list_acc_train = []
list_loss_val = []
list_acc_val = []
list_preds_vec = []
list_true_vec = []

for i in range(2):
    model_ft = initialize_model(len(classes))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model_ft.parameters())

    model_ft = model_ft.to(device)

    model_ft, loss_train, acc_train, loss_val, acc_val = train_model(model_ft, criterion, optimizer,
                       num_epochs=200)
    
    preds_vec, true_vec = prediciton(model_ft)
    
    list_loss_train.append(loss_train)
    list_acc_train.append(acc_train)
    list_loss_val.append(loss_val)
    list_acc_val.append(acc_val)
    list_preds_vec.append(preds_vec)
    list_true_vec.append(true_vec)
    
    torch.save(model_ft, f'Resnet_224_{i}_{date}')
# Save Pickles
results = open(f'Resnet_results_224_{date}', 'wb')
pickle.dump([[list_loss_train],[list_acc_train],[list_loss_val],[list_acc_val],[list_preds_vec],[list_true_vec]],results)
results.close()

Epoch 0/199
----------
Train Loss: 0.9797 Acc: 0.6432
Val Loss: 0.8455 Acc: 0.7390

Epoch 1/199
----------
Train Loss: 0.6748 Acc: 0.7598
Val Loss: 0.7168 Acc: 0.7530

Epoch 2/199
----------
Train Loss: 0.6299 Acc: 0.7705
Val Loss: 0.6311 Acc: 0.7950

Epoch 3/199
----------
Train Loss: 0.5611 Acc: 0.7903
Val Loss: 0.6389 Acc: 0.7750

Epoch 4/199
----------
Train Loss: 0.5692 Acc: 0.7920
Val Loss: 0.6509 Acc: 0.7650

Epoch 5/199
----------
Train Loss: 0.5356 Acc: 0.8023
Val Loss: 0.5998 Acc: 0.7940

Epoch 6/199
----------
Train Loss: 0.5070 Acc: 0.8140
Val Loss: 0.6153 Acc: 0.7880

Epoch 7/199
----------
Train Loss: 0.5180 Acc: 0.8133
Val Loss: 0.5972 Acc: 0.8050

Epoch 8/199
----------
Train Loss: 0.5022 Acc: 0.8130
Val Loss: 0.5971 Acc: 0.7920

Epoch 9/199
----------
Train Loss: 0.4813 Acc: 0.8173
Val Loss: 0.6422 Acc: 0.7580

Epoch 10/199
----------
Train Loss: 0.4872 Acc: 0.8175
Val Loss: 0.6718 Acc: 0.7360

Epoch 11/199
----------
Train Loss: 0.4877 Acc: 0.8200
Val Loss: 0.5693 Acc

Val Loss: 0.5262 Acc: 0.8110

Epoch 97/199
----------
Train Loss: 0.4292 Acc: 0.8433
Val Loss: 0.5177 Acc: 0.8260

Epoch 98/199
----------
Train Loss: 0.4235 Acc: 0.8468
Val Loss: 0.5436 Acc: 0.8170

Epoch 99/199
----------
Train Loss: 0.4169 Acc: 0.8463
Val Loss: 0.5179 Acc: 0.8150

Epoch 100/199
----------
Train Loss: 0.4540 Acc: 0.8340
Val Loss: 0.5105 Acc: 0.8190

Epoch 101/199
----------
Train Loss: 0.4330 Acc: 0.8410
Val Loss: 0.5630 Acc: 0.8060

Epoch 102/199
----------
Train Loss: 0.4387 Acc: 0.8350
Val Loss: 0.5264 Acc: 0.8320

Epoch 103/199
----------
Train Loss: 0.4157 Acc: 0.8430
Val Loss: 0.5119 Acc: 0.8180

Epoch 104/199
----------
Train Loss: 0.4337 Acc: 0.8347
Val Loss: 0.4973 Acc: 0.8260

Epoch 105/199
----------
Train Loss: 0.4571 Acc: 0.8413
Val Loss: 0.5294 Acc: 0.8300

Epoch 106/199
----------
Train Loss: 0.4456 Acc: 0.8333
Val Loss: 0.5645 Acc: 0.8070

Epoch 107/199
----------
Train Loss: 0.4404 Acc: 0.8390
Val Loss: 0.5039 Acc: 0.8300

Epoch 108/199
----------
Tr

Train Loss: 0.4441 Acc: 0.8330
Val Loss: 0.5590 Acc: 0.8110

Epoch 193/199
----------
Train Loss: 0.4475 Acc: 0.8405
Val Loss: 0.5840 Acc: 0.7920

Epoch 194/199
----------
Train Loss: 0.4310 Acc: 0.8433
Val Loss: 0.5130 Acc: 0.8270

Epoch 195/199
----------
Train Loss: 0.4452 Acc: 0.8383
Val Loss: 0.5345 Acc: 0.8150

Epoch 196/199
----------
Train Loss: 0.4273 Acc: 0.8500
Val Loss: 0.5156 Acc: 0.8370

Epoch 197/199
----------
Train Loss: 0.4353 Acc: 0.8427
Val Loss: 0.5251 Acc: 0.8240

Epoch 198/199
----------
Train Loss: 0.4602 Acc: 0.8375
Val Loss: 0.5292 Acc: 0.8180

Epoch 199/199
----------
Train Loss: 0.4378 Acc: 0.8403
Val Loss: 0.5199 Acc: 0.8260

Training complete in 209m 59s
Best val Acc: 0.841000
Epoch 0/199
----------
Train Loss: 0.9748 Acc: 0.6442
Val Loss: 0.8691 Acc: 0.6930

Epoch 1/199
----------
Train Loss: 0.6662 Acc: 0.7670
Val Loss: 0.7613 Acc: 0.7300

Epoch 2/199
----------
Train Loss: 0.5842 Acc: 0.7923
Val Loss: 0.7278 Acc: 0.7270

Epoch 3/199
----------
Train Los

Train Loss: 0.4422 Acc: 0.8410
Val Loss: 0.5334 Acc: 0.8170

Epoch 89/199
----------
Train Loss: 0.4434 Acc: 0.8377
Val Loss: 0.5766 Acc: 0.7890

Epoch 90/199
----------
Train Loss: 0.4583 Acc: 0.8277
Val Loss: 0.5293 Acc: 0.8100

Epoch 91/199
----------
Train Loss: 0.4424 Acc: 0.8367
Val Loss: 0.5430 Acc: 0.8110

Epoch 92/199
----------
Train Loss: 0.4286 Acc: 0.8475
Val Loss: 0.5417 Acc: 0.8030

Epoch 93/199
----------
Train Loss: 0.4641 Acc: 0.8253
Val Loss: 0.5318 Acc: 0.8120

Epoch 94/199
----------
Train Loss: 0.4243 Acc: 0.8420
Val Loss: 0.5081 Acc: 0.8190

Epoch 95/199
----------
Train Loss: 0.4444 Acc: 0.8373
Val Loss: 0.5256 Acc: 0.8210

Epoch 96/199
----------
Train Loss: 0.4403 Acc: 0.8373
Val Loss: 0.5612 Acc: 0.8070

Epoch 97/199
----------
Train Loss: 0.4256 Acc: 0.8433
Val Loss: 0.5106 Acc: 0.8270

Epoch 98/199
----------
Train Loss: 0.4456 Acc: 0.8380
Val Loss: 0.5385 Acc: 0.8040

Epoch 99/199
----------
Train Loss: 0.4465 Acc: 0.8395
Val Loss: 0.5091 Acc: 0.8270

Epoc

Train Loss: 0.4289 Acc: 0.8395
Val Loss: 0.5276 Acc: 0.8290

Epoch 185/199
----------
Train Loss: 0.4406 Acc: 0.8415
Val Loss: 0.5038 Acc: 0.8250

Epoch 186/199
----------
Train Loss: 0.4545 Acc: 0.8355
Val Loss: 0.5838 Acc: 0.7900

Epoch 187/199
----------
Train Loss: 0.4334 Acc: 0.8360
Val Loss: 0.5602 Acc: 0.8100

Epoch 188/199
----------
Train Loss: 0.4216 Acc: 0.8490
Val Loss: 0.5054 Acc: 0.8270

Epoch 189/199
----------
Train Loss: 0.4239 Acc: 0.8433
Val Loss: 0.5466 Acc: 0.8150

Epoch 190/199
----------
Train Loss: 0.4357 Acc: 0.8353
Val Loss: 0.6793 Acc: 0.7380

Epoch 191/199
----------
Train Loss: 0.4455 Acc: 0.8380
Val Loss: 0.5034 Acc: 0.8250

Epoch 192/199
----------
Train Loss: 0.4101 Acc: 0.8498
Val Loss: 0.5538 Acc: 0.8110

Epoch 193/199
----------
Train Loss: 0.4082 Acc: 0.8528
Val Loss: 0.5169 Acc: 0.8170

Epoch 194/199
----------
Train Loss: 0.4162 Acc: 0.8468
Val Loss: 0.5810 Acc: 0.7920

Epoch 195/199
----------
Train Loss: 0.4249 Acc: 0.8453
Val Loss: 0.5058 Acc: 0